In [1]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

In [2]:
from openai_hackathon_wait.tools.arxiv_tool import arxiv_search

In [3]:
PAPER_CONTENT = """
# Abstract

We present PPCEF, a novel method for generating prob-
abilistically plausible counterfactual explanations (CFs). PPCEF ad-
vances beyond existing methods by combining a probabilistic formula-
tion that leverages the data distribution with the optimization of plausi-
bility within a unified framework. Compared to reference approaches,
our method enforces plausibility by directly optimizing the explicit
density function without assuming a particular family of parametrized
distributions. This ensures CFs are not only valid (i.e., achieve class
change) but also align with the underlying data’s probability density.
For that purpose, our approach leverages normalizing flows as power-
ful density estimators to capture the complex high-dimensional data
distribution. Furthermore, we introduce a novel loss function that bal-
ances the trade-off between achieving class change and maintaining
closeness to the original instance while also incorporating a proba-
bilistic plausibility term. PPCEF’s unconstrained formulation allows
for an efficient gradient-based optimization with batch processing,
leading to orders of magnitude faster computation compared to prior
methods. Moreover, the unconstrained formulation of PPCEF allows
for the seamless integration of future constraints tailored to specific
counterfactual properties. Finally, extensive evaluations demonstrate
PPCEF’s superiority in generating high-quality, probabilistically plau-
sible counterfactual explanations in high-dimensional tabular settings.

# Keywords
Counterfactual Explanations, Probabilistic Plausibility, Normalizing Flows, Explainable AI (XAI)
"""

In [4]:
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
import asyncio

arxiv_agent = Agent(
    name="Arxiv agent",
    instructions="You provide information about the papers that are related to the query.",
    tools=[arxiv_search],
    model="gpt-4o-mini",
)

triage_agent = Agent(
    name="Triage agent",
    instructions="You are a helpful assistant that finds the most relevant papers from Arxiv.",
    handoffs=[arxiv_agent],
    model="gpt-4o-mini",
)

In [5]:
async def main():
    result = await Runner.run(
        triage_agent, input="Find papers about soft decision trees in machine learning"
    )
    print(result.final_output)

In [6]:
await main()

It seems there is an issue with accessing the arXiv database for papers on soft decision trees in machine learning. While I can't retrieve that information at the moment, I can suggest the following approach:

1. **Visit arXiv**: You can directly search for "soft decision trees" on [arXiv.org](https://arxiv.org/). Use filters in the Computer Science > Learning category.
2. **Use Google Scholar**: Searching for the same keywords in [Google Scholar](https://scholar.google.com/) can help you find papers and their citations.

If you need more specific information or help with something else, let me know!
